# 第10章 — ニューラルネットワーク

- パーセプトロンは重み付き和に活性化をかけ、層を重ねて関数を近似する。
- 学習では損失の勾配降下で重みを更新する。
- 非線形活性化（ReLUやシグモイド）が曲線的な決定境界を表現する。
- データの正規化と小さめの学習率が安定性を高める。

試してみよう: 活性化をReLUに変えたり、OR/XORを学習して（XORは中間層が必要）挙動を試す。

In [ ]:
import mathdef sigmoid(x):    return 1 / (1 + math.exp(-x))class Perceptron:    def __init__(self, w, b=0.0, lr=0.1):        self.w = w        self.b = b        self.lr = lr    def predict(self, x1, x2):        z = self.w[0] * x1 + self.w[1] * x2 + self.b        return sigmoid(z)    def train(self, data, epochs=20):        for _ in range(epochs):            for x1, x2, label in data:                pred = self.predict(x1, x2)                error = label - pred                self.w[0] += self.lr * error * x1                self.w[1] += self.lr * error * x2                self.b += self.lr * error# AND gate datasetsamples = [    (0, 0, 0),    (0, 1, 0),    (1, 0, 0),    (1, 1, 1),]p = Perceptron([0.1, 0.1], b=-0.1, lr=0.5)p.train(samples, epochs=30)for x1, x2, _ in samples:    print((x1, x2), "->", round(p.predict(x1, x2), 3))